# Imports

In [8]:
import pandas as pd
import numpy as np
from numpy import mean
from numpy import std
import sklearn
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier
from sklearn.ensemble import VotingClassifier

# Train

In [4]:
train_set = pd.read_csv('train/train_encoded.csv')

In [5]:
train_set.drop('id', axis=1, inplace=True)
train_set

,len_keyword,len_location,len_text,len_text_original,diff_len_text,keyword_sum,keyword_min,keyword_max,keyword_mean,keyword_median,...,text_min_tf-idf,text_max_tf-idf,text_mean_tf-idf,text_median_tf-idf,text_sum_w2v,text_min_w2v,text_max_w2v,text_mean_w2v,text_median_w2v,target
0,0.0,0.0,68,69,1,1.0,1.0,1.0,1.0,1.0,...,0.099866,0.430347,0.257765,0.249176,11.683651,-0.018116,0.615265,0.177025,0.150866,1
1,0.0,0.0,38,38,0,1.0,1.0,1.0,1.0,1.0,...,0.227820,0.504613,0.365052,0.347656,0.790154,-0.103024,0.308946,0.079015,0.055061,1
2,0.0,0.0,131,133,2,1.0,1.0,1.0,1.0,1.0,...,0.074545,0.497279,0.232038,0.219919,32.468316,-0.044882,1.000000,0.154611,0.113670,1
3,0.0,0.0,63,65,2,1.0,1.0,1.0,1.0,1.0,...,0.128410,0.484991,0.335797,0.356367,1.943116,-0.039545,0.317247,0.092529,0.078856,1
4,0.0,0.0,86,88,2,1.0,1.0,1.0,1.0,1.0,...,0.096273,0.390788,0.248369,0.241596,13.153737,-0.041156,1.000000,0.125274,0.086994,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7608,0.0,0.0,61,83,22,1.0,1.0,1.0,1.0,1.0,...,0.128512,0.391212,0.307677,0.318035,4.244816,-0.026625,0.312744,0.117912,0.114414,1
7609,0.0,0.0,122,125,3,1.0,1.0,1.0,1.0,1.0,...,0.166141,0.352438,0.238144,0.229751,25.374036,-0.006511,1.000000,0.211450,0.158652,1
7610,0.0,0.0,40,65,25,1.0,1.0,1.0,1.0,1.0,...,0.117186,0.427936,0.321743,0.353433,0.795194,-0.016643,0.215985,0.132532,0.165923,1
7611,0.0,0.0,137,137,0,1.0,1.0,1.0,1.0,1.0,...,0.070977,0.495946,0.222646,0.194716,24.563957,-0.091922,1.000000,0.116971,0.085238,1


In [12]:
X, y = train_set.iloc[:,:-1],train_set.iloc[:,-1]
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

## Bagging

In [10]:
lgbm_model = lgb.LGBMClassifier()

model = BaggingClassifier(base_estimator=lgbm_model)

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=5, random_state=42)
results = model_selection.cross_val_score(model, X_train, y_train, cv=cv, scoring='f1')

print(results.mean())

0.6334029299909184


In [13]:
lgbm_model = lgb.LGBMClassifier(objective='binary', num_leaves=40, learning_rate=0.1, max_depth=100)

model = BaggingClassifier(base_estimator=lgbm_model)

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=5, random_state=42)
results = model_selection.cross_val_score(model, X, y, cv=cv, scoring='f1')

print(results.mean())

0.6438864350632691


**=================**

In [14]:
lgbm_model = lgb.LGBMClassifier(objective='binary', num_leaves=40, learning_rate=0.1, max_depth=100)

model = BaggingClassifier(base_estimator=lgbm_model)

cv = RepeatedStratifiedKFold(random_state=42)
results = model_selection.cross_val_score(model, X, y, cv=cv, scoring='f1')

print(results.mean())

0.6399951290540629


In [15]:
seed = 111
num_trees = 100
kfold = model_selection.KFold(n_splits=2)
model = AdaBoostClassifier(n_estimators=num_trees, random_state=seed)

results = model_selection.cross_val_score(model, X, y, cv=kfold)
print(results.mean())

0.6530932316096092


In [16]:
kfold = model_selection.KFold(n_splits=3)
estimators = []

model1 = lgb.LGBMClassifier(objective='binary', num_leaves=20, learning_rate=0.1)
estimators.append(('lgbm', model1))

model2 = xgb.XGBClassifier(objective ='reg:logistic', colsample_bytree = 0.5,\
                           learning_rate = 0.01, max_depth = 100, n_estimators = 100)
estimators.append(('xgb', model2))

model3 = CatBoostClassifier(silent=True)
estimators.append(('catboost', model3))

model4 = AdaBoostClassifier(n_estimators=100, random_state=42)
estimators.append(('adaboost', model4))

ensemble = VotingClassifier(estimators)
results = model_selection.cross_val_score(ensemble, X, y, cv=kfold)
print(results.mean())

0.6705669669143589


0.6081618329246296

In [17]:
kfold = model_selection.KFold(n_splits=2)
estimators = []

model1 = lgb.LGBMClassifier()
estimators.append(('lgbm', model1))

model2 = xgb.XGBClassifier()
estimators.append(('xgb', model2))

model3 = CatBoostClassifier(silent=True)
estimators.append(('catboost', model3))

ensemble = VotingClassifier(estimators)
results = model_selection.cross_val_score(ensemble, X, y, cv=kfold)
print(results.mean())

0.650072894456529


In [18]:
kfold = model_selection.KFold(n_splits=10)
estimators = []


model4 = AdaBoostClassifier(n_estimators=100, random_state=42)
estimators.append(('adaboost', model4))

model1 = lgb.LGBMClassifier(objective='binary', num_leaves=20, learning_rate=0.1)
estimators.append(('lgbm', model1))

model3 = CatBoostClassifier(silent=True)
estimators.append(('catboost', model3))

model2 = xgb.XGBClassifier(objective ='reg:logistic', colsample_bytree = 0.5,\
                           learning_rate = 0.01, max_depth = 100, n_estimators = 100)
estimators.append(('xgb', model2))

ensemble = VotingClassifier(estimators)
results = model_selection.cross_val_score(ensemble, X, y, cv=kfold)
print(results.mean())

0.6741047316523016


In [19]:
kfold = model_selection.KFold(n_splits=10)
models = []

model4 = AdaBoostClassifier(n_estimators=100, random_state=42)
models.append(('adaboost', model4))

model1 = lgb.LGBMClassifier(objective='binary', num_leaves=20, learning_rate=0.1)
models.append(('lgbm', model1))

model3 = CatBoostClassifier(silent=True)
models.append(('catboost', model3))

model2 = xgb.XGBClassifier(objective ='reg:logistic', colsample_bytree = 0.5,\
                           learning_rate = 0.1, max_depth = 100, n_estimators = 100)
models.append(('xgb', model2))

ensemble = VotingClassifier(models)
results = model_selection.cross_val_score(ensemble, X, y, cv=kfold)
print(results.mean())

0.6714773005542506


In [25]:
kfold = model_selection.KFold(n_splits=10)
models = []

model1 = lgb.LGBMClassifier(objective='binary', num_leaves=20, learning_rate=0.1)
models.append(('lgbm', model1))

model2 = xgb.XGBClassifier(objective ='reg:logistic', colsample_bytree = 0.5,\
                           learning_rate = 0.1, max_depth = 100, n_estimators = 100)
models.append(('xgb', model2))

model3 = CatBoostClassifier(silent=True)
models.append(('catboost', model3))

model4 = AdaBoostClassifier(n_estimators=100, random_state=42)
models.append(('adaboost', model4))

model5 = AdaBoostClassifier(n_estimators=50, random_state=42)
models.append(('adaboost_2', model5))

model6 = xgb.XGBClassifier(objective ='reg:logistic', colsample_bytree = 0.5,\
                           learning_rate = 0.1, max_depth = 100, n_estimators = 57)
models.append(('xgb_2', model6))

model7 = lgb.LGBMClassifier(objective='binary', num_leaves=100, learning_rate=0.1)
models.append(('lgbm_2', model7))

ensemble = VotingClassifier(models)
results = model_selection.cross_val_score(ensemble, X, y, cv=kfold)
print(results.mean())

0.6726608172007409


## Old model

In [27]:
model = BaggingClassifier(base_estimator=lgb.LGBMClassifier(objective='binary', num_leaves=40,\
                                                           learning_rate=0.1, max_depth=100))

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=42)
n_scores = cross_val_score(model, X, y, scoring='f1', cv=cv)

print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))
print(n_scores)

Accuracy: 0.645 (0.022)
[0.67651888 0.65897858 0.64900662 0.64855688 0.61691542 0.62837838
 0.63545151 0.62184874 0.65442404 0.66979656 0.65084746 0.63787375
 0.63513514 0.63476874 0.65874363 0.66034755 0.69508197 0.63497453
 0.58181818 0.64473684 0.65776294 0.62176166 0.63772955 0.66993464
 0.63398693 0.65912763 0.63366337 0.62091503 0.64440735 0.68300654]
